### 📍 요금정보 ['이용요금', '입장료', '상세정보'] : 유료/무료 형태로
    - '유/무료 여부' 컬럼 새로 생성

In [ ]:
# '유/무료 여부' 컬럼을 생성하고, 모든 행을 'X'로 초기화
df_fin = df_fin.withColumn('유/무료 여부', lit('X'))

In [ ]:
### 1. 관광지 분류 == '레포츠'
from pyspark.sql.functions import coalesce

# '결제정보' 컬럼 생성
df_fin = df_fin.withColumn('결제정보', when(col('관광지 분류') == '레포츠', concat_ws(' ', coalesce(col('이용요금').cast('string'), ''), coalesce(col('상세정보'), ''))).otherwise(col('결제정보')))

# '무료' 및 '유료' 여부 확인
keywords = ['무료', r'\d+천원', r'\d+원']
filtered_rows = expr("rlike(결제정보, '{}') AND NOT (rlike(결제정보, '{}') OR rlike(결제정보, '{}'))".format('|'.join(keywords), r'\d+천원', r'\d+원'))

paid_kw = [r'\d+천원', r'\d+원']
paid_rows = expr("rlike(결제정보, '{}') OR rlike(결제정보, '{}')".format(*paid_kw))



# '유/무료 여부' 컬럼 생성
df_fin = df_fin.withColumn('유/무료 여부', when((col('관광지 분류') == '레포츠') & filtered_rows, '무료')
                                .when((col('관광지 분류') == '레포츠') & paid_rows, '유료')
                                .otherwise('알 수 없음'))
# '결제정보' 컬럼 제거
df_fin = df_fin.drop('결제정보')

# 결과 표시
df_fin.select('유/무료 여부').show()


In [ ]:
### 2. 관광지분류 == '인문관광지'

filtered_rows = col('관광지 분류') == '인문관광지'

# '무료'에 해당하는 키워드
free_keywords = ['입 장 료:없음', '입 장 료:무료', '시설이용료:없음', '시설이용료:무료', '입장료 무료', '입장료 없음']

# '유료'에 해당하는 키워드
paid_keywords = [r'\d+천원', r'\d+원']

# '유/무료 여부' 컬럼 설정
df_fin = df_fin.withColumn('유/무료 여부',
                          when(filtered_rows & col('상세정보').isNotNull() & col('상세정보').rlike('|'.join(free_keywords)), '무료')
                          .when(filtered_rows & col('상세정보').isNotNull() & col('상세정보').rlike('|'.join(paid_keywords)), '유료')
                          .otherwise('알 수 없음'))

In [ ]:
### 3. 관광지분류 == '문화시설', '축제공연행사'

# '문화시설' 및 '축제공연행사'에 해당하는 행 필터링
filtered_rows = (col('관광지 분류').isin('문화시설', '축제공연행사'))

# '무료'에 해당하는 키워드
free_keywords = ['무료', '이용요금 없음']

# '유료'에 해당하는 키워드
paid_keywords = [r'\d+천원', r'\d+원', '유료']

# '유/무료 여부' 컬럼 설정
df_fin = df_fin.withColumn('유/무료 여부', 
                          when(filtered_rows & col('이용요금').rlike('|'.join(free_keywords)), '무료')
                          .when(filtered_rows & col('이용요금').rlike('|'.join(paid_keywords)), '유료')
                          .otherwise('알 수 없음'))
